<a href="https://colab.research.google.com/github/Shibu4064/ML_Project/blob/main/HACKANONS_COLAB_25GB_RAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install transformers[torch]
!pip install transformers[tf-cpu]
!pip install transformers[flax]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.6/225.6 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import multiprocessing
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import transformers
import torch.nn as nn
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoModelForMaskedLM, AutoModelForPreTraining, EarlyStoppingCallback
from transformers import AutoTokenizer, AutoConfig
from transformers import BertForMaskedLM, DistilBertForMaskedLM
from transformers import BertTokenizer, DistilBertTokenizer
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from tokenizers import BertWordPieceTokenizer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import gdown

dataset_link = "https://drive.google.com/uc?id=1GyiajN4C6dmk55X_E0BLw8-fFK52ZGYe"

gdown.download(dataset_link,"dataset.csv")

Downloading...
From: https://drive.google.com/uc?id=1GyiajN4C6dmk55X_E0BLw8-fFK52ZGYe
To: /content/dataset.csv
100%|██████████| 31.5M/31.5M [00:01<00:00, 30.5MB/s]


'dataset.csv'

In [5]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")
model = AutoModelForMaskedLM.from_pretrained("csebuetnlp/banglabert")

Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['generator_predictions.dense.weight', 'generator_predictions.LayerNorm.weight', 'generator_lm_head.bias', 'generator_predictions.dense.bias', 'generator_predictions.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model

ElectraForMaskedLM(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,

In [7]:
SEED_SPLIT = 0
SEED_TRAIN = 0

MAX_SEQ_LEN = 128
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 16
LEARNING_RATE = 2e-5
LR_WARMUP_STEPS = 100
WEIGHT_DECAY = 0.01

In [8]:
dtf_mlm = pd.read_csv('/content/dataset.csv')
dtf_mlm = dtf_mlm.rename(columns={"review_content": "text"})

# Train/Valid Split
df_train, df_valid = train_test_split(
    dtf_mlm, test_size=0.15, random_state=SEED_SPLIT
)

len(df_train), len(df_valid)

# Convert to Dataset object
train_dataset = Dataset.from_pandas(df_train[['text']].dropna())
valid_dataset = Dataset.from_pandas(df_valid[['text']].dropna())

In [9]:
def tokenize_function(row):
    return tokenizer(
        row['text'],
        padding='max_length',
        truncation=True,
        max_length=MAX_SEQ_LEN,
        return_special_tokens_mask=True)

column_names = train_dataset.column_names

train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=multiprocessing.cpu_count(),
    remove_columns=column_names,
)

valid_dataset = valid_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=multiprocessing.cpu_count(),
    remove_columns=column_names,
)

Map (num_proc=2):   0%|          | 0/79656 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/14058 [00:00<?, ? examples/s]

In [10]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [11]:
from transformers import  Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy, AutoModelForSequenceClassification


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


steps_per_epoch = int(len(train_dataset) / TRAIN_BATCH_SIZE)

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/EMNLP/buetcsenlp_bert_finetuing1/',
    logging_dir='/content/drive/MyDrive/EMNLP/buetcsenlp_bert_finetuing1/log',
    num_train_epochs=10,
    do_train=True,
    do_eval=True,
    eval_steps=250,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    warmup_steps=LR_WARMUP_STEPS,
    save_steps=steps_per_epoch,
    save_total_limit=3,
    weight_decay=WEIGHT_DECAY,
    learning_rate=LEARNING_RATE,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
    seed=SEED_TRAIN,
    # save_total_limit = 5
)

# training_args = TrainingArguments(
#    f"training_with_callbacks",
#    evaluation_strategy = IntervalStrategy.STEPS, # "steps"
#   #  output_dir= "/content/drive/MyDrive/EMNLP/buetcsenlp_bert_finetuning_1",
#    eval_steps = 250, # Evaluation and Save happens every 250 steps
#    save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
#   #  save_steps = steps_per_epoch,
#    learning_rate=2e-5,
#    per_device_train_batch_size=1,
#    per_device_eval_batch_size=1,
#    num_train_epochs=5,
#    weight_decay=0.01,
#    metric_for_best_model = 'loss',
#    load_best_model_at_end=True)

trainer = Trainer(
    model=model,
    args=training_args,
    # compute_metrics=compute_metrics,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

In [16]:
!gdown 1VkaYtWD_5DT1X1IqqeJXp5NPdt56l3-5 -O checkpoint.zip

Downloading...
From: https://drive.google.com/uc?id=1VkaYtWD_5DT1X1IqqeJXp5NPdt56l3-5
To: /content/checkpoint.zip
100% 1.22G/1.22G [00:14<00:00, 84.1MB/s]


In [17]:
!unzip checkpoint.zip

Archive:  checkpoint.zip
   creating: last training checkpoint for finetuning/
  inflating: last training checkpoint for finetuning/optimizer.pt  
  inflating: last training checkpoint for finetuning/config.json  
  inflating: last training checkpoint for finetuning/trainer_state.json  
  inflating: last training checkpoint for finetuning/scheduler.pt  
  inflating: last training checkpoint for finetuning/tokenizer_config.json  
  inflating: last training checkpoint for finetuning/tokenizer.json  
  inflating: last training checkpoint for finetuning/vocab.txt  
  inflating: last training checkpoint for finetuning/rng_state.pth  
  inflating: last training checkpoint for finetuning/special_tokens_map.json  
  inflating: last training checkpoint for finetuning/pytorch_model.bin  
  inflating: last training checkpoint for finetuning/training_args.bin  


In [18]:
trainer.train("/content/last training checkpoint for finetuning")

You are resuming training from a checkpoint trained with 4.31.0 of Transformers but your current version is 4.32.0. This is not recommended and could yield to errors or unwanted behaviors.


Epoch,Training Loss,Validation Loss
9,3.486900,3.438241
10,3.431300,3.436887


TrainOutput(global_step=49790, training_loss=0.6963063617192208, metrics={'train_runtime': 4520.0301, 'train_samples_per_second': 176.229, 'train_steps_per_second': 11.015, 'total_flos': 5.261765538270413e+16, 'train_loss': 0.6963063617192208, 'epoch': 10.0})

In [19]:
trainer.save_model("/content/drive/MyDrive/EMNLP/fine_tuned_large_bangla_bert1.pth")